In [3]:
import pandas as pd
import requests

# Load your Excel file
df = pd.read_excel("D:/AAAAAAAA/pincode.xlsx")

# Example format of the file:
# | Pincode | Locality | Landmark | Road | Colony | Marg |
# Make sure to rename columns if needed to match your structure


ModuleNotFoundError: No module named 'requests'

# Geocode Using Geoapify

In [ ]:
API_KEY = "YOUR_GEOAPIFY_API_KEY"  # Replace with your Geoapify API key

def geocode_address(address):
    url = "https://api.geoapify.com/v1/geocode/search"
    params = {
        "text": address,
        "apiKey": API_KEY,
        "format": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data.get("features"):
        coords = data["features"][0]["geometry"]["coordinates"]
        return coords[1], coords[0]  # (latitude, longitude)
    return None, None


#  Build Full Address and Get Coordinates

In [ ]:
def build_address(row):
    parts = [str(row.get(col, "")) for col in ['Locality', 'Landmark', 'Road', 'Colony', 'Marg', 'Pincode']]
    return ", ".join([part for part in parts if part])

df["Full_Address"] = df.apply(build_address, axis=1)
df[["Latitude", "Longitude"]] = df["Full_Address"].apply(lambda addr: pd.Series(geocode_address(addr)))


#  Route Mapping Between Two Points

In [ ]:
def get_route(lat1, lon1, lat2, lon2):
    url = "https://api.geoapify.com/v1/routing"
    params = {
        "waypoints": f"{lat1},{lon1}|{lat2},{lon2}",
        "mode": "drive",
        "apiKey": API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    if "features" in data:
        route = data["features"][0]["properties"]
        return route["distance"], route["time"]
    return None, None


#  Example: Route from First to Second Row

In [ ]:
lat1, lon1 = df.iloc[0]["Latitude"], df.iloc[0]["Longitude"]
lat2, lon2 = df.iloc[1]["Latitude"], df.iloc[1]["Longitude"]

distance, time = get_route(lat1, lon1, lat2, lon2)
print(f"Distance: {distance/1000:.2f} km, Time: {time/60:.2f} mins")
